In [1]:
from urllib.request import urlopen
import datetime
from datetime import timedelta
import csv
import pandas as pd
from scipy import signal
from matplotlib import pyplot as plt
import numpy as np
import holidays
import os
from datetime import timedelta



#function that receives market sales for each market date and calculates the sales average of previous markets

#inputs: input list-->market sales (ordered in ascending time)
            #moving average: number of market sales prior to a given market date to average
    
            #output vector that consists of the average of (#moving_average) number of market dates prior to a given market date
def return_prev_avg(input_list,moving_average):
    hit=0
    previous_markets=[]
    for i,item in enumerate(input_list):
        if i>=moving_average:
            previous_markets.append(np.mean(input_list[i-moving_average:i]))

        else:
            previous_markets.append(item)
    return previous_markets

In [2]:
#read revenue files for each agricultural market vendor and calculate previous sales, market name, other propoerties, etc.

os.chdir('C:\\Users\\teek2\\Documents\\Revenue_Data_Files')

AG_markets=['SAT','SUN','WED','PICO']
AG_files=["Sat_AG.pkl","Sun_AG.pkl","Wed_AG.pkl","Pico_AG.pkl"]
AG_dfs=[]

for i,file in enumerate(AG_files):
    dummy_df=pd.read_pickle(file)
    dummy_df['Previous Markets']=return_prev_avg(list(dummy_df["Revenue"]),4)
    dummy_df['Market Name']=AG_markets[i]
    dummy_df['Market Product']='AG'
    dummy_df['Num Vendors']=1
    AG_dfs.append(dummy_df)
    
total_AG=pd.concat(AG_dfs)

In [3]:
#do the same for other vendors

CP_markets=['SUN','PICO']
CP_files=["Sun_CP.pkl","Pico_CP.pkl"]
vendors=[['Hot Food','Baked Goods','Coffee','Retail','Pre-packaged Food'],['Hot Food','Baked Goods','Coffee','Pre-packaged Food']]

In [4]:
cp_df=[]

for i,file in enumerate(CP_files):
    cp_temp_df=pd.read_pickle(file)
    for current_vendor in vendors[i]:
        dummy_df=pd.DataFrame()
        
        if CP_markets[i]=='SUN':
            dummy_df['Date']=cp_temp_df['Date'].apply(lambda x: x.date())
        else:
            dummy_df['Date']=cp_temp_df['Date']
            
        dummy_df['Revenue']=cp_temp_df[current_vendor+"_Average"]
        dummy_df['Previous Markets']=return_prev_avg(list(cp_temp_df[current_vendor+"_Average"]),4)
        dummy_df['Market Product']=current_vendor
        dummy_df['Market Name']=CP_markets[i]
        dummy_df['Num Vendors']=cp_temp_df[current_vendor+"_Count"]
        cp_df.append(dummy_df)
        
total_cf=pd.concat(cp_df)
   

In [5]:
total_rev_df=pd.concat([total_cf,total_AG])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
#save output file

total_rev_df.to_pickle('Market_revenues.pkl')